## Graph Base Data Coding Tasks

1. SNAP data statistics and visualization  
2. CORA data exploration 

## 1. SNAP Exploration

Goals 
- Visualize Degree Distribution
- Create Adjacency Matrix, then visualize connections 
- Calculate Average Path Length
- With Adjacency Matrix, observe connectedness
- Calculate Cluster Coefficient for the graph 
- Visualize the Graph with networkx

In [19]:
import networkx as nx
import numpy as np 
import pandas as pd
import json

Start with Twitch Data

In [51]:
twitch_edges = pd.read_csv("../data/raw/snap/twitch/ENGB/musae_ENGB_edges.csv") # get edges, skip 1 to skip header
twitch_target = pd.read_csv("../data/raw/snap/twitch/ENGB/musae_ENGB_target.csv") # get target
# get features
with open("../data/raw/snap/twitch/ENGB/musae_ENGB_features.json", "r") as file: 
    twitch_features = json.loads(file.read())

In [52]:
# collect the nodes 
twitch_nodes = twitch_target.new_id.to_numpy(dtype='int')
twitch_nodes[:10]

array([2299,  153,  397, 5623, 5875, 3346, 6930, 6367, 1434, 4470])

In [53]:
# reformat dataframe into array for faster speed
twitch_edges = twitch_edges.to_numpy(dtype='int')
twitch_edges[:5]

array([[6194,  255],
       [6194,  980],
       [6194, 2992],
       [6194, 2507],
       [6194,  986]])

In [54]:
# get some basic stats on the nodes 
def general_graph_stats(edges, nodes):
    """Generate a printed report of general graph statistics"""
    print(f"There are {len(nodes)} nodes.")
    print(f"There are {edges.shape[0]} edges.")
    print(f"Average Degree.")

general_graph_stats(twitch_edges, twitch_nodes)

There are 7126 nodes.
There are 35324 edges.
Average Degree.


## 2. CORA Exploration 

Goals
- Download the Cora dataset. Read the readme file.
- Construct a graph from the features (word_attributes within cora.content) using one of the methods you learned about in section 2.2 of the von Luxburg tutorial in week 1. You may use sklearn.neighbors in your implementation.
- Construct an adjacency matrix from the citation information given in cora.cites.
- Compare the graph properties of the constructed graph to the given citation graph. Are the graphs similar?

In [3]:
import sklearn.neighbors as nb
import numpy as np